In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
#data = pd.read_csv('winereviews.csv')
#data.head()
column_headers = ["country", "description","points", "province", "region_1", "variety", "winery"]
data = pd.read_csv('winereviews.csv', usecols=column_headers)

In [3]:
data.head()

,country,description,points,province,region_1,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,96,California,Napa Valley,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",96,Northern Spain,Toro,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,96,California,Knights Valley,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",96,Oregon,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",95,Provence,Bandol,Provence red blend,Domaine de la Bégude


In [4]:
data = data[data.winery.notnull()]
data = data[data.variety.notnull()]
data = data[data.province != "Other"]
data = data[data.province.notnull()]
data = data[data.province != "France Other"]
data = data[data.province != "Spain Other"]
data = data[data.province != "Australia Other"]
data = data[data.country != "US-France"]
data = data[data.points > 87]
data = data.reset_index()

In [5]:
len(data.country.unique())

37

In [6]:
data.shape

(78673, 8)

In [7]:
reviews = []
for review in data['description']:
    reviews.append([review])
document = []
for review in data['description']:
    document.append(review)
document = ' '.join(document)
document = document.lower()
document = re.sub('[^a-zA-Z]', ' ', document )
re.sub(r'\s+', ' ', document)
document[:100]

'this tremendous      varietal wine hails from oakville and was aged over three years in oak  juicy r'

In [8]:
all_sentences = nltk.sent_tokenize(document)
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

In [9]:
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if (w not in stopwords.words('english')) and w not in ['wine', 'drink', 'flavors', 'vineyard', 'finish', 'palate', 'drink', 'notes', 'very']]

In [10]:
from gensim.models import Word2Vec

In [11]:
word2vec = Word2Vec(all_words, min_count = 2, window = 5, size = 300)

In [12]:
vocabulary = word2vec.wv.vocab
word2vec.wv.most_similar('peach')

[('aromas', 0.760880708694458),
 ('tannins', 0.7596301436424255),
 ('black', 0.7564570903778076),
 ('juicy', 0.7455992698669434),
 ('pepper', 0.7453985810279846),
 ('acidity', 0.7446411848068237),
 ('fruit', 0.7438744306564331),
 ('apple', 0.7437282800674438),
 ('nose', 0.7436293363571167),
 ('berry', 0.7228865623474121)]

In [13]:
query = ['dry']

In [14]:
### new
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from gensim.models.phrases import Phrases, Phraser

In [15]:
reviews = list(data['description'])
reviews = [str(r) for r in reviews]
reviews_full_doc = ' '.join(reviews)
sent_tokens = sent_tokenize(reviews_full_doc)

In [16]:
#remove stop words, lowercase, and punctuation
punc = '''!()-[]{};:.'"\, <>./?@#$%^&*_~'''

def process_text(text):
    words = word_tokenize(text)
    processed_sentence = []
    for word in words:
        w = str(word)
        lowercase = str.lower(w)
        stemmed = SnowballStemmer('english').stem(lowercase)
        remove_punc = stemmed
        for c in remove_punc:
            if c in punc:
                remove_punc.replace(c, "")
        processed_sentence.append(remove_punc)
    return(processed_sentence)
        
processed_sents = []
for sent in sent_tokens:
    processed = process_text(sent)
    processed_sents.append(processed)

In [17]:
# add bi grams/phrases
phrases = Phrases(processed_sents)
phrases = Phrases(phrases[processed_sents])
grams = Phraser(phrases)
phrase_sents = []
for sent in processed_sents:
    phrased = grams[sent]
    phrase_sents.append(phrased)
    
allwords = [item for sublist in phrase_sents for item in sublist]

In [18]:
allwords

['this',
 'tremend',
 '100_%',
 'variet',
 'wine',
 'hail_from',
 'oakvill',
 'and',
 'was',
 'age',
 'over',
 'three_year',
 'in',
 'oak',
 '.',
 'juici',
 'red-cherri',
 'fruit',
 'and',
 'a',
 'compel',
 'hint',
 'of',
 'caramel',
 'greet',
 'the',
 'palat',
 ',',
 'frame',
 'by',
 'eleg',
 ',',
 'fine',
 'tannin',
 'and',
 'a',
 'subtl',
 'minti',
 'tone',
 'in',
 'the',
 'background',
 '.',
 'balanc',
 'and',
 'reward',
 'from',
 'start',
 'to',
 'finish',
 ',',
 'it',
 'has',
 'year',
 'ahead',
 'of',
 'it',
 'to',
 'develop',
 'further',
 'nuanc',
 '.',
 'enjoy',
 '2022–2030',
 '.',
 'ripe',
 'aroma',
 'of',
 'fig',
 ',',
 'blackberri',
 'and',
 'cassi',
 'are',
 'soften',
 'and',
 'sweeten',
 'by',
 'a',
 'slather',
 'of',
 'oaki',
 'chocol',
 'and',
 'vanilla',
 '.',
 'this',
 'is',
 'full',
 ',',
 'layer',
 ',',
 'intens',
 'and',
 'cushion',
 'on',
 'the',
 'palat',
 ',',
 'with',
 'rich',
 'flavor',
 'of',
 'chocolati',
 'black',
 'fruit',
 'and',
 'bake',
 'spice',
 '.',
 

In [19]:
mapping = pd.read_csv('descriptor_mapping.csv').set_index('raw descriptor')

In [20]:
mapping.head()

,level_3,level_2,level_1
raw descriptor,,,
abras,abrasive,high_tannin,tannin
acacia,acacia,flowery,flower
acacia_flower,acacia,flowery,flower
aciddriven,acid_driven,high_acid,acid
aggress,aggressive,high_acid,acid


In [21]:
#map corpus words to level 3
def mapped_words(word):
    if word in list(mapping.index):
        mapped = mapping['level_3'][word]
        return mapped
    else:
        return word

processed_sents = []
for sent in phrase_sents:
    processed_sent = []
    for word in sent:
        mapped_word = mapped_words(word)
        processed_sent.append(str(mapped_word))
    processed_sents.append(processed_sent)

In [22]:
processed_sents

[['this',
  'tremend',
  '100_%',
  'variet',
  'wine',
  'hail_from',
  'oakvill',
  'and',
  'was',
  'age',
  'over',
  'three_year',
  'in',
  'oak',
  '.'],
 ['juicy',
  'red-cherri',
  'fruit',
  'and',
  'a',
  'compel',
  'hint',
  'of',
  'caramel',
  'greet',
  'the',
  'palat',
  ',',
  'frame',
  'by',
  'elegant',
  ',',
  'fine',
  'tannin',
  'and',
  'a',
  'subtl',
  'mint',
  'tone',
  'in',
  'the',
  'background',
  '.'],
 ['balanc',
  'and',
  'reward',
  'from',
  'start',
  'to',
  'finish',
  ',',
  'it',
  'has',
  'year',
  'ahead',
  'of',
  'it',
  'to',
  'develop',
  'further',
  'nuanc',
  '.'],
 ['enjoy', '2022–2030', '.'],
 ['ripe',
  'aroma',
  'of',
  'fig',
  ',',
  'blackberry',
  'and',
  'cassis',
  'are',
  'soften',
  'and',
  'sweet',
  'by',
  'a',
  'slather',
  'of',
  'oak',
  'chocolate',
  'and',
  'vanilla',
  '.'],
 ['this',
  'is',
  'full',
  ',',
  'layer',
  ',',
  'intens',
  'and',
  'cushion',
  'on',
  'the',
  'palat',
  ',',
 

In [23]:
#embeddings:

In [24]:
model = Word2Vec(processed_sents, size=300, min_count=1, iter=15)

In [25]:
print(model)

Word2Vec(vocab=31060, size=300, alpha=0.025)


In [26]:
print(model)

Word2Vec(vocab=31060, size=300, alpha=0.025)


In [27]:
model.wv.most_similar(positive='peach', topn=10)

[('guava', 0.6767099499702454),
 ('honeydew', 0.6759825348854065),
 ('nectarine', 0.6671782732009888),
 ('tangerine', 0.6659562587738037),
 ('lemon_curd', 0.6650081872940063),
 ('quince', 0.6564744114875793),
 ('banana', 0.6479943990707397),
 ('pear', 0.6449276208877563),
 ('mango', 0.6326952576637268),
 ('stone-fruit', 0.6262353658676147)]

In [28]:
#update review data to only show meaningful words
reviews = list(data['description'])
mapped_reviews = []
def mapping_wine_words(word):
    if word in list(mapping.index):
        mapped = mapping['level_3'][word]
        return mapped
for review in reviews:
    processed = process_text(review)
    phrased = grams[processed]
    mapped = [mapping_wine_words(word) for word in phrased]
    remove_none = [str(d) for d in mapped if d is not None]
    mapped_rev = ' '.join(remove_none)
    mapped_reviews.append(mapped_rev)

In [29]:
mapped_reviews

['oak juicy fruit caramel elegant mint',
 'ripe fig blackberry cassis sweet oak chocolate vanilla rich chocolate fruit baked spice toast heady',
 'complex white dark gold toasted_hazelnut pear compote orange_peel succulent residual_sugar',
 'oak fruit dense toast toast cigar_box blackberry cherry coffee graphite polished dark chocolate coffee',
 'dense wood rich concentrated',
 'depth dense dark ripe fruit oak blackberry coffee coffee toast oak spice smooth heady',
 'gritty sweet pastry prune blackberry coffee chocolate',
 'lush cedar almond vanilla tannin lush chocolate baked berry cassis hard tannin depth',
 'minerality chunky fruit citrus graphite concentrated',
 'bright crunchy cranberry pomegranate orange_peel silky succulent fleshy fruit lush length',
 'elegant complex white yellow spring_flower herb fruit cream juicy peach ripe pear citrus white almond savory minerality',
 'supple coffee cherry vanilla savory',
 'green cola tobacco orange vanilla tea milk_chocolate ripe blackber

In [30]:
## create embedding vectors for each review
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit(mapped_reviews)
dict_of_tfidf_weightings = dict(zip(X.get_feature_names(), X.idf_))

In [32]:
dict_of_tfidf_weightings

{'abrasive': 9.054192186465674,
 'acacia': 7.563537810021541,
 'aggressive': 6.645446897643238,
 'airy': 7.9423346710475435,
 'allspice': 7.27585573756976,
 'almond': 4.920626911090292,
 'alpine_herbs': 8.94086350115867,
 'american_oak': 7.545680192621534,
 'angular': 7.303254711757874,
 'anise': 4.95519181270738,
 'apple': 3.6578410796462717,
 'apple_sauce': 10.663630098899775,
 'apricot': 4.37058057417102,
 'ash': 7.719191119733334,
 'asian_spice': 6.338173815714287,
 'asparagus': 8.230016743499323,
 'asphalt': 6.893170657793415,
 'assertive': 6.542968228360301,
 'astringent': 5.3916567076913395,
 'austere': 5.598506619519449,
 'bacon': 5.388581359291092,
 'baked': 4.701079561956435,
 'baked_apple': 6.542968228360301,
 'baked_bread': 7.55456914003878,
 'balsamic': 5.734928187566204,
 'banana': 6.429523594302515,
 'barbecue': 6.627621113690637,
 'bark': 7.493944518222345,
 'barnyard': 7.27585573756976,
 'basil': 8.422920409623817,
 'bay_leaf': 7.996401892317819,
 'beef_jerky': 7.36041

In [33]:
review_vectors = []
for review in mapped_reviews:
    numwords = 0
    words = review.split(' ')
    tfidf_mapped_review = []
    for word in words:
        if word in dict_of_tfidf_weightings.keys():
            tfidf = dict_of_tfidf_weightings[word]
            vec = tfidf * model.wv.get_vector(word).reshape(1, 300)
            tfidf_mapped_review.append(vec)
            numwords = numwords + 1
    if len(tfidf_mapped_review) > 0:
        weight_rev_vec = sum(tfidf_mapped_review)/len(tfidf_mapped_review)
    else:
        weight_rev_vec = []
    review_vectors.append(weight_rev_vec)

In [34]:
print(model.wv)

In [35]:
query = ['peach']

In [36]:
import numpy as np
vectors_list = []
for vec in review_vectors:
    if len(vec) == 1:
        vectors_list.append(np.float32(vec[0]))
    else:
        vectors_list.append(np.float32(np.zeros(300)))

In [37]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10, algorithm= 'brute', metric='cosine')
modelknn = knn.fit(vectors_list)

In [38]:
weightedqueryterms = []
for term in query:
    if term in dict_of_tfidf_weightings:
        tfidfweight = dict_of_tfidf_weightings[term]
        word_vector = tfidfweight * model.wv.get_vector(term).reshape(1,300)
        weightedqueryterms.append(word_vector)
    else:
        try:
            print(term)
            word_vector = model.wv.get_vector(term)
            word_vector.shape(1,300)
            print(word_vector)
            weightedqueryterms.append(word_vector)
        except:
            print("???")
query_vec = sum(weightedqueryterms)
distance, indice = modelknn.kneighbors(query_vec, n_neighbors = 5)

In [39]:
distance_list = distance[0].tolist()[1:]
indice_list = indice[0].tolist()[1:]

In [40]:
distance_list

[0.1326150894165039,
 0.13486415147781372,
 0.1362687349319458,
 0.14575928449630737]

In [41]:
indice_list

[36100, 17883, 55921, 21821]

In [42]:
for i in indice_list:
    print(i)
    review = mapped_reviews[i]
    location = data['province'][i]
    wine = data['variety'][i]
    print(location)
    print(wine)
    print(review)
    print()

36100
Rheingau
Riesling
mango peach honey peach nectar citrus rich

17883
Western Australia
Chardonnay
bread citrus peach plump restrained peach pineapple lime

55921
Galicia
Albariño
racy peach nectarine apple peach lime easy

21821
Oregon
Chardonnay
peach hazelnut savory crisp fruit peach pear yellow apple



In [43]:
#new word embedding tfidf matrix of reviews
np.array(vectors_list).shape

(78673, 300)

In [44]:
#cleaned and mapped reviews (with only wine words)
mapped_reviews

['oak juicy fruit caramel elegant mint',
 'ripe fig blackberry cassis sweet oak chocolate vanilla rich chocolate fruit baked spice toast heady',
 'complex white dark gold toasted_hazelnut pear compote orange_peel succulent residual_sugar',
 'oak fruit dense toast toast cigar_box blackberry cherry coffee graphite polished dark chocolate coffee',
 'dense wood rich concentrated',
 'depth dense dark ripe fruit oak blackberry coffee coffee toast oak spice smooth heady',
 'gritty sweet pastry prune blackberry coffee chocolate',
 'lush cedar almond vanilla tannin lush chocolate baked berry cassis hard tannin depth',
 'minerality chunky fruit citrus graphite concentrated',
 'bright crunchy cranberry pomegranate orange_peel silky succulent fleshy fruit lush length',
 'elegant complex white yellow spring_flower herb fruit cream juicy peach ripe pear citrus white almond savory minerality',
 'supple coffee cherry vanilla savory',
 'green cola tobacco orange vanilla tea milk_chocolate ripe blackber

In [45]:
data['description'] = mapped_reviews
data.drop(columns = 'index')
wine_dict = data.to_dict(orient='index')

In [46]:
wine_dict

{0: {'index': 0,
  'country': 'US',
  'description': 'oak juicy fruit caramel elegant mint',
  'points': 96,
  'province': 'California',
  'region_1': 'Napa Valley',
  'variety': 'Cabernet Sauvignon',
  'winery': 'Heitz'},
 1: {'index': 1,
  'country': 'Spain',
  'description': 'ripe fig blackberry cassis sweet oak chocolate vanilla rich chocolate fruit baked spice toast heady',
  'points': 96,
  'province': 'Northern Spain',
  'region_1': 'Toro',
  'variety': 'Tinta de Toro',
  'winery': 'Bodega Carmen Rodríguez'},
 2: {'index': 2,
  'country': 'US',
  'description': 'complex white dark gold toasted_hazelnut pear compote orange_peel succulent residual_sugar',
  'points': 96,
  'province': 'California',
  'region_1': 'Knights Valley',
  'variety': 'Sauvignon Blanc',
  'winery': 'Macauley'},
 3: {'index': 3,
  'country': 'US',
  'description': 'oak fruit dense toast toast cigar_box blackberry cherry coffee graphite polished dark chocolate coffee',
  'points': 96,
  'province': 'Oregon',

In [47]:
model.wv

In [48]:
import pickle

In [49]:
# with open('word2vecmodel.pickle', 'wb') as handle:
#     pickle.dump(model.wv, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
# with open('mapped_reviews.pickle', 'wb') as handle:
#     pickle.dump(mapped_reviews, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
with open('tfidf_weight_dict.pickle', 'wb') as handle:
    pickle.dump(dict_of_tfidf_weightings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
# with open('review_vectors_list.pickle', 'wb') as handle:
#     pickle.dump(vectors_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
# with open('knnmodel.pickle', 'wb') as handle:
#     pickle.dump(modelknn, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
print(model)

Word2Vec(vocab=31060, size=300, alpha=0.025)


In [55]:
matrix_word2vec = []
words_word2vec_dict = {}
for i, key in enumerate(list(model.wv.vocab.keys())):
    words_word2vec_dict[key] = i
    embedding = model.wv.get_vector(key)
    matrix_word2vec.append(embedding)
matrix_word2vec = np.array(matrix_word2vec)
matrix_word2vec.shape

(31060, 300)

In [56]:
from scipy import sparse

In [57]:
matrix_word2vec = sparse.csr_matrix(matrix_word2vec)

In [58]:
vectors_list = np.array(vectors_list)
vectors_list_sparse = sparse.csr_matrix(vectors_list)

In [72]:
with open('matrix_word2vec.pickle', 'wb') as handle:
    pickle.dump(matrix_word2vec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [73]:
with open('words_word2vec.pickle', 'wb') as handle:
    pickle.dump(words_word2vec_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
#vectors_list = np.float32(vectors_list)
with open('review_tfidf_embeddings.pickle', 'wb') as handle:
    pickle.dump(vectors_list_sparse, handle, protocol=pickle.HIGHEST_PROTOCOL)#

In [74]:
country_to_idx = {}
for idx in wine_dict:
    c = wine_dict[idx]['country']
    if c not in country_to_idx:
        country_to_idx[c] = []
    country_to_idx[c].append(idx)

with open('country_to_idx.pickle', 'wb') as handle:
    pickle.dump(country_to_idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
with open('wine_dict.pickle', 'wb') as handle:
    pickle.dump(wine_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [76]:
with open('tfidf_weight_dict.pickle', 'wb') as handle:
    pickle.dump(dict_of_tfidf_weightings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [65]:
type(vectors_list)

numpy.ndarray

In [66]:
type(vectors_list_sparse)

scipy.sparse.csr.csr_matrix

In [67]:
#import os
#os.path.getsize('C:\\Python27\\Lib\\genericpath.py')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Python27\\Lib\\genericpath.py'

In [68]:
vectors_list

array([[-1.766482  , -3.8173764 ,  1.3883544 , ...,  4.609987  ,
        -1.1379833 ,  2.8260686 ],
       [-2.693226  , -1.7496849 ,  2.6203828 , ...,  3.5077353 ,
        -0.2981766 ,  5.206307  ],
       [-1.2526373 , -4.019906  ,  0.05989867, ...,  2.9655287 ,
         2.038269  ,  4.031085  ],
       ...,
       [-0.6842202 , -5.716008  ,  2.0319638 , ...,  3.2959113 ,
        -1.8486694 ,  3.6718056 ],
       [-1.444748  , -1.8099562 ,  0.49009892, ...,  2.3621411 ,
        -1.1539477 ,  1.8195015 ],
       [-0.79453546, -1.1528265 , -0.60718966, ...,  1.5190315 ,
         1.3912673 ,  2.049505  ]], dtype=float32)

In [69]:
vectors_list_sparse.data.nbytes

94392000

In [70]:
with open('review_tfidf_embeddings.pickle', 'wb') as handle:
    pickle.dump(vectors_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
matrix_word2vec.data.nbytes

37272000